<a href="https://colab.research.google.com/github/jaideep99/GFA-Predictor/blob/main/PGFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import re
import pickle
import math
import tensorflow as tf

In [11]:
# melting-point, enthalphy, electro-negativity, atomic-radius of elemnts
pm = {'Ag': {'mp': 961, 'enthalphy': 11.3, 'en': 1.93, 'ar': 1.72}, 'Al': {'mp': 660.25, 'enthalphy': 10.7, 'en': 1.61, 'ar': 1.43}, 'Au': {'mp': 1064, 'enthalphy': 12.5, 'en': 2.54, 'ar': 1.66}, 'B': {'mp': 2076, 'enthalphy': 50.0, 'en': 2.04, 'ar': 0.85}, 'Be': {'mp': 1287, 'enthalphy': 7.95, 'en': 1.57, 'ar': 1.12}, 'C': {'mp': 3500, 'enthalphy': 105.0, 'en': 2.55, 'ar': 0.7}, 'Ca': {'mp': 839, 'enthalphy': 8.53, 'en': 1.0, 'ar': 2.31}, 'Ce': {'mp': 798, 'enthalphy': 5.5, 'en': 1.12, 'ar': 2.48}, 'Co': {'mp': 1495, 'enthalphy': 16.2, 'en': 1.88, 'ar': 2.0}, 'Cr': {'mp': 1857, 'enthalphy': 20.5, 'en': 1.66, 'ar': 1.28}, 'Cu': {'mp': 1084.6, 'enthalphy': 13.1, 'en': 1.9, 'ar': 1.4}, 'Dy': {'mp': 1412, 'enthalphy': 11.1, 'en': 1.22, 'ar': 2.35}, 'Er': {'mp': 1522, 'enthalphy': 19.9, 'en': 1.24, 'ar': 2.32}, 'Fe': {'mp': 1535, 'enthalphy': 13.8, 'en': 1.83, 'ar': 1.26}, 'Ga': {'mp': 29.76, 'enthalphy': 5.59, 'en': 1.81, 'ar': 1.87}, 'Gd': {'mp': 1312, 'enthalphy': 10.0, 'en': 1.2, 'ar': 1.8}, 'Hf': {'mp': 2227, 'enthalphy': 25.5, 'en': 1.3, 'ar': 2.25}, 'Ho': {'mp': 1470, 'enthalphy': 17.0, 'en': 1.23, 'ar': 2.33}, 'In': {'mp': 156.76, 'enthalphy': 3.26, 'en': 1.78, 'ar': 1.56}, 'La': {'mp': 920, 'enthalphy': 6.3, 'en': 1.1, 'ar': 2.5}, 'Mg': {'mp': 649, 'enthalphy': 8.7, 'en': 1.31, 'ar': 1.73}, 'Mn': {'mp': 1244, 'enthalphy': 13.2, 'en': 1.55, 'ar': 1.61}, 'Mo': {'mp': 2617, 'enthalphy': 36.0, 'en': 2.16, 'ar': 1.9}, 'Nb': {'mp': 2468, 'enthalphy': 26.8, 'en': 1.6, 'ar': 2.15}, 'Nd': {'mp': 1016, 'enthalphy': 7.1, 'en': 1.14, 'ar': 2.29}, 'Ni': {'mp': 1453, 'enthalphy': 17.2, 'en': 1.91, 'ar': 1.63}, 'P': {'mp': 44.3, 'enthalphy': 0.64, 'en': 2.19, 'ar': 1.95}, 'Pd': {'mp': 1552, 'enthalphy': 16.0, 'en': 2.2, 'ar': 1.63}, 'Pr': {'mp': 931, 'enthalphy': 6.9, 'en': 1.13, 'ar': 2.47}, 'Sc': {'mp': 1541, 'enthalphy': 16.0, 'en': 1.36, 'ar': 2.3}, 'Si': {'mp': 1414, 'enthalphy': 50.2, 'en': 1.9, 'ar': 1.11}, 'Sn': {'mp': 231.9, 'enthalphy': 7.0, 'en': 1.96, 'ar': 2.25}, 'Ta': {'mp': 3017, 'enthalphy': 36.0, 'en': 1.5, 'ar': 2.2}, 'Ti': {'mp': 1668, 'enthalphy': 18.7, 'en': 1.54, 'ar': 1.47}, 'Tm': {'mp': 1545, 'enthalphy': 16.8, 'en': 1.25, 'ar': 2.3}, 'Y': {'mp': 1526, 'enthalphy': 11.4, 'en': 1.22, 'ar': 2.4}, 'Zn': {'mp': 419.73, 'enthalphy': 7.35, 'en': 1.65, 'ar': 1.39}, 'Zr': {'mp': 1852, 'enthalphy': 21.0, 'en': 1.33, 'ar': 2.3}}

In [12]:
# seperates big-small atoms
def diff(x):

  #making ranges for each atom
  ranges = {}
  for i in x:
    ranges[i] = 0.88*pm[i]['ar']
  
  #compiling scoring matrix
  score = {}
  
  for i in x:
    dct = {}
    for j in x:
      if pm[i]['ar']<ranges[j]:
        dct[j] = -1
      elif pm[i]['ar']>pm[j]['ar']:
        dct[j] = 1
      else :
        dct[j] = 0
          
    score[i] = dct
    
  b = []
  sm = []
  print()
  #separating into big and small based on scoring matrix
  for i in score:
    sum = 0
    for j in score[i]:
      sum = sum+score[i][j]
      
    if(sum>0):
      b.append(i)
      
    else : 
      sm.append(i)
      
      
  return b,sm

In [13]:
#finds the paramater deltaE
def electro(all,x):
  k =0 #summation of product of composition and electro negativity
  p = 0 #summation of compositions

  for i in all:
    if i in x:
      p = p+all[i]
  for i in all:
    if i in x:
      k = k+all[i]*pm[i]['en']
  
  return k/p

In [14]:
#finds the paramater deltaD
def comps(all,x):
  k =0 #summation of product of composition and atomic radii
  p = 0 #summation of compositions

  for i in all:
    if i in x:
      p = p+all[i]
  for i in all:
    if i in x:
      k = k+all[i]*pm[i]['ar']
  
  return k/p

In [15]:
def prepare_params(x):
  #Seperating atoms from composition
  s = re.sub(r'[^\w\s]','',x)
  s = re.sub('\d',' ',s)
  els = np.array([i for i in s.split(' ') if i in pm])
  print('\nElements in BMG are : ', els)
  
  comp = re.findall(r"[-+]?\d*\.\d+|\d+",x)
  comp = [float(i) for i in comp]
  print('Compositions: ',comp)
  
  dcomp = dict(zip(els,comp))
  smix = 0
  hmix = 0

  for i in dcomp:
    smix = smix + (dcomp[i]/100)*(math.log((dcomp[i]/100)))
    hmix = hmix + (dcomp[i]/100)*pm[i]['enthalphy']
  smix = -1*smix

  big,small = diff(els)

  maxrad = 0.5137797338363198
  maxen = 0.1441441441441441
  rad = (comps(dcomp,big)-comps(dcomp,small))/(comps(dcomp,big))
  rad = int((rad/maxrad)*100)
  en = (electro(dcomp,big)-electro(dcomp,small))/(electro(dcomp,big)+electro(dcomp,small))
  en = int((en/maxen)*100)

  return [hmix,smix,rad,en]

In [20]:
# Loads ANN model and applies it to predict GFA of given composition
def predictGfaANN(bmg):
  model = tf.keras.models.load_model('drop_model_final.h5')
  inp = np.array([bmg])
  pred = model.predict(inp)
  return pred[0][0]

In [31]:
# Loads RF model and applies it to predict GFA of given composition
def predictGfaRF(bmg):
  rf_model = pickle.load(open('random_forest.sav', 'rb'))
  inp = np.array([bmg])
  pred = rf_model.predict(inp)
  return pred[0]

In [28]:
# Applies pre-computed standard scaler on params/input
def scaleParams(bmg):
  scaler = pickle.load(open('st_scaler.sav', 'rb'))
  bmg = np.array([bmg])
  params_scaled = scaler.transform(bmg)
  return params_scaled[0]

In [32]:
bmg = input("Enter BMG elements and compositions : ")

params = prepare_params(bmg)


print("Computed Parameters are : ",params)

params = scaleParams(params)
print("Computed Scaled Parameters are : ",params)


GFA_ANN = predictGfaANN(params)
GFA_RF = predictGfaRF(params)

print("\nPredicted Dmax ANN: ",GFA_ANN)
print("\nPredicted Dmax ANN: ",GFA_RF)

Enter BMG elements and compositions : Pd73.5Cu10Si16.5

Elements in BMG are :  ['Pd' 'Cu' 'Si']
Compositions:  [73.5, 10.0, 16.5]

Computed Parameters are :  [21.353, 0.7538524402682971, 49, 50]
Computed Scaled Parameters are :  [ 1.0567944  -1.51735497 -1.15588644  2.3168874 ]

Predicted Dmax ANN:  1.8465838

Predicted Dmax ANN:  2.28125
